# CODTECH Internship - Data Science
## Task 4: Advanced Optimization Model: Multi-Product Mix Optimization

### Problem Description

This task presents an advanced business optimization challenge: determining the optimal production quantities for a factory manufacturing multiple products to maximize overall profit. This model considers several resource constraints and market demand limitations, making it a more realistic scenario.

**Scenario:**
A factory produces three distinct products: Product A, Product B, and Product C. The objective is to establish the ideal weekly production volume for each product to achieve the highest possible total profit, while adhering to available machine time, raw material supply, and labor hours, as well as individual product market demands.

**Resource Requirements and Profit Contribution per Unit:**

| Product | Machine Time (hrs/unit) | Raw Material (kg/unit) | Labor Hours (hrs/unit) | Profit ($/unit) | Max Demand (units/week) |
| :------ | :---------------------- | :--------------------- | :--------------------- | :-------------- | :---------------------- |
| **A** | 3                       | 1                      | 2                      | 12              | 40                      |
| **B** | 2                       | 2                      | 3                      | 10              | 35                      |
| **C** | 4                       | 1                      | 2                      | 15              | 30                      |

**Available Resources (Weekly):**
* Total Machine Time: 200 hours
* Total Raw Material: 150 kg
* Total Labor Hours: 180 hours

**Objective:**
Determine the optimal number of units of Product A, Product B, and Product C to produce each week to maximize the total profit, given the specified resource and demand constraints.

### Mathematical Formulation

This problem is formulated as a Linear Programming (LP) model to find the optimal solution.

**1. Decision Variables:**
These represent the unknown quantities we need to determine, which are the weekly production volumes for each product.
* Let $x_A$: represent the number of units of Product A to produce per week.
* Let $x_B$: represent the number of units of Product B to produce per week.
* Let $x_C$: represent the number of units of Product C to produce per week.

*(All decision variables must be non-negative integers: $x_A, x_B, x_C \in \mathbb{Z}_{\ge 0}$)*

**2. Objective Function:**
The goal is to maximize the total profit, $Z$. This is calculated by summing the profit contributions of all products.
* **Maximize $Z = 12x_A + 10x_B + 15x_C$**

**3. Constraints:**
These are the limitations imposed by the available resources and market demand.

* **Machine Time Constraint:** The total machine time utilized by all products must not exceed 200 hours.
    $3x_A + 2x_B + 4x_C \le 200$

* **Raw Material Constraint:** The total raw material consumed by all products must not exceed 150 kg.
    $1x_A + 2x_B + 1x_C \le 150$

* **Labor Hours Constraint:** The total labor hours expended on all products must not exceed 180 hours.
    $2x_A + 3x_B + 2x_C \le 180$

* **Demand Constraint (Product A):** Production of Product A cannot exceed its maximum weekly demand of 40 units.
    $x_A \le 40$

* **Demand Constraint (Product B):** Production of Product B cannot exceed its maximum weekly demand of 35 units.
    $x_B \le 35$

* **Demand Constraint (Product C):** Production of Product C cannot exceed its maximum weekly demand of 30 units.
    $x_C \le 30$

* **Non-negativity and Integer Constraints:** Production quantities must be non-negative and whole numbers.
    $x_A \ge 0, x_B \ge 0, x_C \ge 0$
    $x_A, x_B, x_C \in \mathbb{Z}$

In [ ]:
# --- CODTECH Internship Task 4: Optimization Model ---
# Problem: Enhanced Product Mix Optimization for a Factory

# Import necessary libraries
from pulp import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Problem Definition ---
# Create the LP problem object - maximize total profit
prob = LpProblem("Advanced_Factory_Product_Mix_Optimization", LpMaximize)

# --- 2. Define Decision Variables ---
# Using a dictionary to manage variables for scalability
products = ['A', 'B', 'C']
product_vars = LpVariable.dicts("Product", products, lowBound=0, cat=LpInteger)

# --- 3. Define Parameters (Data) ---
# Profit per unit
profit = {
    'A': 12,
    'B': 10,
    'C': 15
}

# Resource consumption per unit for each product
machine_time_req = {
    'A': 3,
    'B': 2,
    'C': 4
}
raw_material_req = {
    'A': 1,
    'B': 2,
    'C': 1
}
labor_hours_req = {
    'A': 2,
    'B': 3,
    'C': 2
}

# Available resources
max_machine_time = 200
max_raw_material = 150
max_labor_hours = 180

# Maximum demand for each product
max_demand = {
    'A': 40,
    'B': 35,
    'C': 30
}

# --- 4. Define Objective Function ---
# Maximize: 12*A + 10*B + 15*C
prob += lpSum(profit[i] * product_vars[i] for i in products), "Total_Profit"

# --- 5. Define Constraints ---

# Machine Time Constraint
# 3A + 2B + 4C <= 200
prob += lpSum(machine_time_req[i] * product_vars[i] for i in products) <= max_machine_time, "Machine_Time_Constraint"

# Raw Material Constraint
# 1A + 2B + 1C <= 150
prob += lpSum(raw_material_req[i] * product_vars[i] for i in products) <= max_raw_material, "Raw_Material_Constraint"

# Labor Hours Constraint
# 2A + 3B + 2C <= 180
prob += lpSum(labor_hours_req[i] * product_vars[i] for i in products) <= max_labor_hours, "Labor_Hours_Constraint"

# Demand Constraints for each product
for p in products:
    prob += product_vars[p] <= max_demand[p], f"Demand_Constraint_{p}"

# --- 6. Solve the Problem ---
print("Solving the optimization problem...")
prob.solve()
print("Problem solved.\n")

# --- 7. Display Results ---

print(f"Solver Status: {LpStatus[prob.status]}\n")

# Only proceed with results if an optimal solution was found
if LpStatus[prob.status] == "Optimal":
    print("--- Optimal Production Plan ---")
    production_plan = {}
    for product in products:
        prod_value = product_vars[product].varValue
        production_plan[product] = prod_value
        print(f"{product_vars[product].name}: {prod_value:.0f} units")

    max_profit = value(prob.objective)
    print(f"\nMaximum Total Profit: ${max_profit:.2f}\n")

    # --- 8. Analyze Resource Utilization and Insights ---

    print("--- Resource Utilization and Insights ---")

    # Calculate actual usage of each resource based on the optimal plan
    actual_machine_time_used = sum(machine_time_req[i] * product_vars[i].varValue for i in products)
    actual_raw_material_used = sum(raw_material_req[i] * product_vars[i].varValue for i in products)
    actual_labor_hours_used = sum(labor_hours_req[i] * product_vars[i].varValue for i in products)

    print(f"Machine Time Used: {actual_machine_time_used:.2f} hours / {max_machine_time} hours available")
    print(f"Raw Material Used: {actual_raw_material_used:.2f} kg / {max_raw_material} kg available")
    print(f"Labor Hours Used: {actual_labor_hours_used:.2f} hours / {max_labor_hours} hours available")

    print("\nKey Business Insights (Binding Constraints & Slack):")

    resources_data = { # Renamed to avoid conflict with 'resources' list later if needed
        "Machine Time": {"used": actual_machine_time_used, "max": max_machine_time, "constraint_obj": prob.constraints["Machine_Time_Constraint"]},
        "Raw Material": {"used": actual_raw_material_used, "max": max_raw_material, "constraint_obj": prob.constraints["Raw_Material_Constraint"]},
        "Labor Hours": {"used": actual_labor_hours_used, "max": max_labor_hours, "constraint_obj": prob.constraints["Labor_Hours_Constraint"]}
    }

    binding_resources = []
    non_binding_resources = []

    for name, data in resources_data.items():
        slack = data["max"] - data["used"]
        if abs(slack) < 0.001: # Use a small tolerance for floating point comparisons
            print(f"- **{name} is a Binding Constraint (Bottleneck):** All {data['max']} units are fully utilized.")
            binding_resources.append(name)
        else:
            print(f"- **{name} has Slack:** {slack:.2f} units remaining. Not fully utilized.")
            non_binding_resources.append(name)

    # --- 9. Sensitivity Analysis (Shadow Prices / Dual Values) ---
    print("\n--- Sensitivity Analysis (Shadow Prices) ---")
    print("Shadow Price (Dual Value) indicates how much the total profit would increase if one additional unit of a *binding* resource were available.")

    shadow_prices = {}
    for name, data in resources_data.items():
        # Directly access .pi; it will be None if not binding, or a float if binding.
        # We only print for binding constraints where .pi is meaningful.
        shadow_price = data["constraint_obj"].pi
        if shadow_price is not None and abs(shadow_price) > 1e-6: # Check if not None and not near zero
            shadow_prices[name] = shadow_price
            print(f"- {name}: ${shadow_price:.2f} per additional unit")
        elif name in binding_resources: # For binding resources with zero shadow price (e.g., degenerate solution)
             print(f"- {name}: ${0.00:.2f} per additional unit (Binding but no marginal profit increase, possibly degenerate solution)")
        else:
            print(f"- {name}: $0.00 per additional unit (Not a binding constraint)")


    print("\n--- Recommendations for Strategic Decision Making ---")
    if len(binding_resources) > 0:
        print(f"To potentially increase profit, the factory should prioritize efforts to increase the availability of its binding resources: {', '.join(binding_resources)}.")
        if len(binding_resources) == len(resources_data):
            print("Since all primary resources are bottlenecks, consider a holistic capacity expansion strategy.")
        else:
            # Sort binding resources by shadow price to prioritize
            prioritized_resources = sorted(binding_resources, key=lambda x: shadow_prices.get(x, 0), reverse=True)
            if prioritized_resources:
                print(f"Specifically, focus on: **{prioritized_resources[0]}** (highest value per additional unit).")
    else:
        print("- The current optimal plan does not fully utilize all resources. This indicates excess capacity. Consider exploring higher market demand, diversifying products, or optimizing for other objectives (e.g., minimizing costs).")

    # --- 10. Visualization of Resource Utilization ---
    print("\n--- Visualization: Resource Utilization ---")

    resource_names = list(resources_data.keys())
    used_values = [data["used"] for data in resources_data.values()]
    max_values = [data["max"] for data in resources_data.values()]

    df_resources = pd.DataFrame({
        'Resource': resource_names,
        'Used': used_values,
        'Available': max_values
    })

    fig, ax = plt.subplots(figsize=(10, 6))

    bar_width = 0.35
    index = range(len(resource_names))

    bar1 = ax.bar(index, df_resources['Available'], bar_width, label='Available Capacity', color='lightgray')
    bar2 = ax.bar([i + bar_width for i in index], df_resources['Used'], bar_width, label='Used Capacity', color='skyblue')

    ax.set_xlabel('Resource')
    ax.set_ylabel('Quantity')
    ax.set_title('Resource Utilization: Used vs. Available Capacity')
    ax.set_xticks([i + bar_width / 2 for i in index])
    ax.set_xticklabels(df_resources['Resource'])
    ax.legend()
    ax.grid(axis='y', linestyle='--', alpha=0.7)

    # Add text labels for used capacity
    for i, used in enumerate(used_values):
        ax.text(i + bar_width, used + 5, f'{used:.2f}', ha='center', va='bottom', fontsize=9, color='blue')

    plt.tight_layout()
    plt.show()

else:
    print("Problem did not solve to optimality. Cannot display results or insights.")

print("\n--- End of Advanced Optimization Analysis ---")